In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import shutil
import zipfile
import matplotlib.pyplot as plt
import numpy as np
import math
from PIL import Image
from google.colab import drive
drive.mount('/content/drive')
%matplotlib inline

Mounted at /content/drive


In [2]:
!kaggle datasets download -d ravyardiankusuma/dataset-motif-batik
!kaggle datasets download -d edywinarno/semarang-batik-dataset

with zipfile.ZipFile("dataset-motif-batik.zip", "r") as zip_ref:
    zip_ref.extractall("dataset-motif-batik")
with zipfile.ZipFile("semarang-batik-dataset.zip", "r") as zip_ref:
    zip_ref.extractall("semarang-batik-dataset")

Dataset URL: https://www.kaggle.com/datasets/ravyardiankusuma/dataset-motif-batik
License(s): DbCL-1.0
100% 2.72G/2.73G [00:28<00:00, 158MB/s]
100% 2.73G/2.73G [00:28<00:00, 101MB/s]
Dataset URL: https://www.kaggle.com/datasets/edywinarno/semarang-batik-dataset
License(s): CC-BY-NC-SA-4.0
100% 12.0G/12.0G [02:27<00:00, 108MB/s] 
100% 12.0G/12.0G [02:27<00:00, 87.7MB/s]


In [3]:
zip_files = ["dataset-motif-batik.zip", "semarang-batik-dataset.zip"]

for zip_file in zip_files:
    if os.path.exists(zip_file):
        os.remove(zip_file)
        print(f"The file '{zip_file}' was successfully deleted.")
    else:
        print(f"The file '{zip_file}' was not found.")

The file 'dataset-motif-batik.zip' was successfully deleted.
The file 'semarang-batik-dataset.zip' was successfully deleted.


In [4]:
base_dir = "motif-batik-indonesia-fix"
base1_dir = "dataset-motif-batik"
base2_dir = "semarang-batik-dataset"

In [5]:
# Rename Folder
for folder_name in os.listdir(base2_dir):
    old_folder_path = os.path.join(base2_dir, folder_name)
    if os.path.isdir(old_folder_path) and "Batik Cap" in folder_name:
        new_folder_name = f"JawaTengah_{folder_name.replace('Batik Cap ', '').replace(' ', '')}"
        new_folder_path = os.path.join(base2_dir, new_folder_name)
        os.rename(old_folder_path, new_folder_path)
        print(f"Folder '{folder_name}' renamed to '{new_folder_name}'")

Folder 'Batik Cap Gambang Semarangan' renamed to 'JawaTengah_GambangSemarangan'
Folder 'Batik Cap Asem Warak' renamed to 'JawaTengah_AsemWarak'
Folder 'Batik Cap Tugu Muda' renamed to 'JawaTengah_TuguMuda'
Folder 'Batik Cap Warak Beras Utah' renamed to 'JawaTengah_WarakBerasUtah'
Folder 'Batik Cap Asem Sinom' renamed to 'JawaTengah_AsemSinom'
Folder 'Batik Cap Blekok Warak' renamed to 'JawaTengah_BlekokWarak'
Folder 'Batik Cap Asem Arang' renamed to 'JawaTengah_AsemArang'
Folder 'Batik Cap Blekok' renamed to 'JawaTengah_Blekok'
Folder 'Batik Cap Semarangan' renamed to 'JawaTengah_Semarangan'
Folder 'Batik Cap Kembang Sepatu' renamed to 'JawaTengah_KembangSepatu'


In [6]:
# Rezise Images
def resize_images_in_folder(folder_path, target_size=(512, 512)):
    valid_extensions = ('.jpg', '.jpeg', '.png', '.JPG', '.JPEG', '.PNG')
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        if file_name.endswith(valid_extensions):
            try:
                img = Image.open(file_path)
                img = img.resize(target_size, Image.Resampling.LANCZOS)
                img.save(file_path)
            except Exception as e:
                print(f"Error resizing {file_path}: {e}")

for folder_name in os.listdir(base2_dir):
    folder_path = os.path.join(base2_dir, folder_name)
    if os.path.isdir(folder_path):
        resize_images_in_folder(folder_path)

print("All images resized successfully.")

All images resized successfully.


In [7]:
folders_to_resize = [
    "JawaTengah_Cipratan", "JawaTengah_IkanKerang", "JawaTengah_JagungLombok",
    "JawaTengah_JambuBelimbing", "JawaTengah_JambuCitra", "JawaTengah_Laut",
    "JawaTengah_LurikSemangka", "JawaTengah_MasjidAgungDemak", "JawaTengah_Naga",
    "JawaTengah_TebuBambu", "JawaTengah_Tembakau", "JawaTengah_Yuyu",
    "Lampung_Bledheg", "Lampung_KacangHijau"
]

for folder_name in folders_to_resize:
    folder_path = os.path.join(base1_dir, folder_name)
    if os.path.isdir(folder_path):
        resize_images_in_folder(folder_path)

print("All images resized successfully.")

All images resized successfully.


In [8]:
# Move Folder
valid_extensions = ('.jpg', '.jpeg', '.png', '.JPG', '.JPEG', '.PNG')

os.makedirs(base_dir, exist_ok=True)

for source_dir in [base1_dir, base2_dir]:
    for folder_name in os.listdir(source_dir):
        source_folder_path = os.path.join(source_dir, folder_name)
        destination_folder_path = os.path.join(base_dir, folder_name)

        if os.path.isdir(source_folder_path):
            os.makedirs(destination_folder_path, exist_ok=True)
            for file_name in os.listdir(source_folder_path):
                if file_name.endswith(valid_extensions):
                    shutil.move(
                        os.path.join(source_folder_path, file_name),
                        os.path.join(destination_folder_path, file_name),
                    )

    shutil.rmtree(source_dir)

print("All valid files moved successfully.")

All valid files moved successfully.


In [9]:
# Augmentation
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest',
    brightness_range=[0.8, 1.2]
)

valid_extensions = ('.jpg', '.jpeg', '.png', '.JPG', '.JPEG', '.PNG')
target_count = 1000

def rename_and_augment(folder_path, folder_name):
    base_name = folder_name.split('_', 1)[-1]

    file_list = [f for f in os.listdir(folder_path) if f.endswith(valid_extensions)]
    print(f"Renaming files in folder '{folder_name}'...")
    for i, file in enumerate(sorted(file_list), start=1):
        old_file_path = os.path.join(folder_path, file)
        file_ext = os.path.splitext(file)[1]
        new_file_name = f"{base_name}_{i:04d}{file_ext}"
        new_file_path = os.path.join(folder_path, new_file_name)
        os.rename(old_file_path, new_file_path)
    print(f"Files in folder '{folder_name}' renamed.")

    file_list = sorted([f for f in os.listdir(folder_path) if f.endswith(valid_extensions)])
    current_count = len(file_list)

    if current_count >= target_count:
        print(f"Folder '{folder_name}' has sufficient files. No augmentation needed.")
        return

    augment_per_file = math.ceil((target_count - current_count) / current_count)
    print(f"Folder '{folder_name}': {current_count} files, augmenting {augment_per_file} times per file.\n")

    file_index = current_count + 1
    for file in file_list:
        img = Image.open(os.path.join(folder_path, file)).convert('RGB')
        img_array = np.expand_dims(np.array(img), axis=0)
        for i, batch in enumerate(datagen.flow(img_array, batch_size=1)):
            if file_index > target_count:
                break

            new_file_name = f"{base_name}_{file_index:04d}.jpg"
            save_path = os.path.join(folder_path, new_file_name)
            Image.fromarray(np.uint8(batch[0])).save(save_path)
            file_index += 1

            if i + 1 >= augment_per_file:
                break

# Augmentation folder
for folder_name in sorted(os.listdir(base_dir)):
    folder_path = os.path.join(base_dir, folder_name)
    if os.path.isdir(folder_path):
        rename_and_augment(folder_path, folder_name)

print("Augmentation completed!")

Renaming files in folder 'Bali_Barong'...
Files in folder 'Bali_Barong' renamed.
Folder 'Bali_Barong': 42 files, augmenting 23 times per file.

Renaming files in folder 'Bali_Merak'...
Files in folder 'Bali_Merak' renamed.
Folder 'Bali_Merak': 37 files, augmenting 27 times per file.

Renaming files in folder 'Jakarta_OndelOndel'...
Files in folder 'Jakarta_OndelOndel' renamed.
Folder 'Jakarta_OndelOndel': 55 files, augmenting 18 times per file.

Renaming files in folder 'Jakarta_Tumpal'...
Files in folder 'Jakarta_Tumpal' renamed.
Folder 'Jakarta_Tumpal': 118 files, augmenting 8 times per file.

Renaming files in folder 'JawaBarat_Megamendung'...
Files in folder 'JawaBarat_Megamendung' renamed.
Folder 'JawaBarat_Megamendung': 349 files, augmenting 2 times per file.

Renaming files in folder 'JawaTengah_AsemArang'...
Files in folder 'JawaTengah_AsemArang' renamed.
Folder 'JawaTengah_AsemArang': 300 files, augmenting 3 times per file.

Renaming files in folder 'JawaTengah_AsemSinom'...
F

In [10]:
zip_file_name = "motif-batik-indonesia-fix.zip"

shutil.make_archive(base_name=zip_file_name.replace(".zip",""), format='zip', root_dir='/content/motif-batik-indonesia-fix')

print(f"ZIP file successfully created: {zip_file_name}")

ZIP file successfully created: motif-batik-indonesia-fix.zip


In [11]:
destination_path = '/content/drive/My Drive/' + zip_file_name
shutil.copy(zip_file_name, destination_path)

print(f"File successfully copied to Google Drive at: {destination_path}")

File successfully copied to Google Drive at: /content/drive/My Drive/motif-batik-indonesia-fix.zip
